In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession
  .builder
  .getOrCreate())  

In [3]:
# Read Data From Sql Server 
jdbcUrl = "jdbc:sqlserver://DESKTOP-L3KHVFJ\SQLEXPRESS:1433;database=businesslocation;trustServerCertificate=true;"
tablename_list=['locationtime','businesstime','registeredbusiness','location']
reader = (
        spark.read.format('jdbc')
        .option("url",jdbcUrl)
        .option('user','username')
        .option('password','password')
)
for tablename in tablename_list:
    reader.option("dbtable",tablename).load().createTempView(tablename)

In [4]:
spark.sql("select * from locationtime limit 5 ").show(5)

+-------------------+----+---+----+-----+----+--------+
|location_start_date|hour|day|week|month|year| weekday|
+-------------------+----+---+----+-----+----+--------+
|2009-11-01 00:00:00|   0|  1|  44|   11|2009|  Sunday|
|2016-05-09 00:00:00|   0|  9|  19|    5|2016|  Monday|
|2014-01-24 00:00:00|   0| 24|   4|    1|2014|  Friday|
|2007-09-24 00:00:00|   0| 24|  39|    9|2007|  Monday|
|2020-08-01 00:00:00|   0|  1|  31|    8|2020|Saturday|
+-------------------+----+---+----+-----+----+--------+



In [7]:
spark.sql("select year,count(*) from locationtime  group by year").show(5)

+----+--------+
|year|count(1)|
+----+--------+
|1959|       1|
|1990|      21|
|1975|      14|
|1977|       9|
|2003|      58|
+----+--------+
only showing top 5 rows



In [9]:
spark.sql("select * from businesstime limit 5").show()

+-------------------+----+---+----+-----+----+--------+
|business_start_date|hour|day|week|month|year| weekday|
+-------------------+----+---+----+-----+----+--------+
|1973-01-02 00:00:00|   0|  2|   1|    1|1973| Tuesday|
|1968-10-01 00:00:00|   0|  1|  40|   10|1968| Tuesday|
|2000-03-06 00:00:00|   0|  6|  10|    3|2000|  Monday|
|1982-08-16 00:00:00|   0| 16|  33|    8|1982|  Monday|
|1998-01-01 00:00:00|   0|  1|   1|    1|1998|Thursday|
+-------------------+----+---+----+-----+----+--------+



In [10]:
spark.sql("select year , count(*) from businesstime group by year").show(5)

+----+--------+
|year|count(1)|
+----+--------+
|1959|       1|
|1990|      30|
|1975|      14|
|1977|      13|
|2003|      69|
+----+--------+
only showing top 5 rows



In [12]:
spark.sql("select * from registeredbusiness limit 5").show(2)

+--------------------+-----------------------+--------------------+-----------+-----------------------+--------------------+--------------------+-------------------+-------------------+
|            uniqueid|business_account_number|      ownership_name|parking_tax|transient_occupancy_tax|   business_location|       location_name|business_start_date|location_start_date|
+--------------------+-----------------------+--------------------+-----------+-----------------------+--------------------+--------------------+-------------------+-------------------+
|0080280-05-001-00...|                  80280|Wollborg-Michelso...|      false|                  false|POINT (-122.408 3...|  Wollborg-Michelson|1973-01-02 00:00:00|2009-11-01 00:00:00|
|1077369-06-161-00...|                  22415|Pflueger & Baerwa...|      false|                  false|POINT (-122.40095...|Pflueger & Baerwa...|1968-10-01 00:00:00|2016-05-09 00:00:00|
+--------------------+-----------------------+--------------------+---

In [19]:
spark.sql("select r.location_name ,count(l.year)\
            from registeredbusiness as r inner join  locationtime as l \
            on  r.location_start_date = l.location_start_date group by r.location_name\
            having count(l.year) > 2").show()

+--------------------+-----------+
|       location_name|count(year)|
+--------------------+-----------+
|          Ecruiz Inc|          3|
|  Hpm Of America Llc|          3|
| Jordie Welles Works|          3|
|Aramark Rfs At Sa...|         14|
|             Propark|          6|
|Exela Enterprise ...|          4|
|       Phil'z Coffee|          6|
|           Lees Deli|          5|
|     Donna's Tamales|          5|
|   Cdf Labor Law Llp|          3|
|The Athletic Medi...|          3|
|       San Francisco|          7|
|           Tcgsf Inc|          3|
|Bon Appetit @ Twi...|          4|
|             Compass|          3|
|            Kim Tsui|          3|
|         Diane Gibbs|          3|
| Hello San Francisco|          3|
|          Wee Scotty|          3|
|         Enforce Llc|          3|
+--------------------+-----------+
only showing top 20 rows



In [20]:
spark.sql("select * from location").show()

+--------------------+--------------------+-------------+-----+--------------+
|       location_name|      street_address|         city|state|source_zipcode|
+--------------------+--------------------+-------------+-----+--------------+
|  Wollborg-Michelson|291 Geary St Ste 500|San Francisco|   CA|         94102|
|Pflueger & Baerwa...| 220 Sansome St 1090|San Francisco|   CA|         94104|
|    Walgreens #15567|       845 Market St|San Francisco|   CA|         94103|
|       Vericlaim Inc|500 Sansome St St...|San Francisco|   CA|         94111|
|Ati Restoration, Llc|25000 Industrial ...|      Hayward|   CA|         94545|
|   Missirlian Donald|500 Sutter St Ste...|San Francisco|   CA|         94102|
|Intermodal Capita...|505 Montgomery St...|San Francisco|   CA|         94111|
| B & J Manufacturing|   1320 Van Dyke Ave|San Francisco|   CA|         94124|
|239-245 Bartlett ...|239 Bartlett St #245|San Francisco|   CA|         94110|
|         Impark 0347|  900 North Point St|San Franc